In [ ]:
from os import getcwd
import os.path as op
from glob import glob

In [ ]:
from clients.utils import create_allocation_table
from hardware.plates import create_plates_from_allocation_table

expt_name = 'A81_E217'
allocation_table = create_allocation_table(expt_name)
expt_plates = create_plates_from_allocation_table(allocation_table)

In [ ]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E217_1'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir))
data_folder = op.join(nav_root, 'vanilla', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
id_qpcr_inst_plate = qpcr_data.get_data_by_well()

In [ ]:
from hardware.labchip import LabChip

lc_plate = '20180103_B'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_inst_plate = lc.get_data_by_well()

In [ ]:
from clients.expt_recipes.common.model_builders import (build_id_qpcr_constituents,
                                                        build_labchip_datas_from_inst_data)
from clients.expt_recipes.vanilla.models import IdConstituents
from clients.expt_recipes.vanilla.model_builders import (build_id_qpcr_datas_from_inst_data,
                                                         create_vanilla_groupings,
                                                         get_wells_by_id_assay)
from clients.expt_recipes.common.utils import create_id_qpcr_lc_mapping, get_lc_dilutions
# from clients.expt_recipes.nested.utils import (get_id_qpcr_plate_names,
#                                                get_labchip_plate_names,
#                                                create_id_qpcr_lc_mapping)
# from clients.expt_recipes.nested.models import NestedMasterTable

In [ ]:
%%time
id_plate_constituents = build_id_qpcr_constituents(
    expt_plates[id_qpcr_plate], expt_plates, IdConstituents)

wells_by_id_assay = get_wells_by_id_assay(id_plate_constituents)

id_qpcr_datas = build_id_qpcr_datas_from_inst_data(id_plate_constituents,
                                                   id_qpcr_inst_plate)

In [ ]:
%%time
mapping = create_id_qpcr_lc_mapping(expt_plates[lc_plate])
dilutions = get_lc_dilutions(expt_plates[lc_plate])
lc_datas = build_labchip_datas_from_inst_data(
    id_plate_constituents, lc_inst_plate, mapping, assays, dilutions)